In [ ]:
import streamlit as st
import pandas as pd
import networkx as nx
from pyvis.network import Network
import random
from datetime import datetime, timedelta
import tempfile
import os

# --- Synthetic Data Generation ---
def generate_synthetic_data(num_partners=30):
    levels = ['Distributor', 'Agent', 'Ambassador']
    data = []
    # Root node
    data.append({
        'partner_id': 1,
        'name': 'Root Distributor',
        'level': 'Distributor',
        'parent_id': None,
        'join_date': (datetime.now() - timedelta(days=random.randint(100, 1000))).strftime('%Y-%m-%d')
    })
    for i in range(2, num_partners + 1):
        level = random.choices(levels, weights=[0.2, 0.5, 0.3])[0]
        parent_candidates = [d for d in data if levels.index(d['level']) < levels.index(level)]
        parent = random.choice(parent_candidates) if parent_candidates else data[0]
        data.append({
            'partner_id': i,
            'name': f'Partner {i}',
            'level': level,
            'parent_id': parent['partner_id'],
            'join_date': (datetime.now() - timedelta(days=random.randint(1, 900))).strftime('%Y-%m-%d')
        })
    return pd.DataFrame(data)

# --- Streamlit App ---
st.set_page_config(page_title="Partner Hierarchy Network", layout="wide")
st.title("Partner Hierarchy Network Visualization")

if 'df' not in st.session_state:
    st.session_state.df = generate_synthetic_data()

if st.button("Regenerate Synthetic Dataset"):
    st.session_state.df = generate_synthetic_data()

df = st.session_state.df

# --- Sidebar Filters ---
levels = df['level'].unique().tolist()
selected_levels = st.sidebar.multiselect("Filter by Level", levels, default=levels)
search_name = st.sidebar.text_input("Search Partner by Name")

filtered_df = df[df['level'].isin(selected_levels)]
if search_name:
    filtered_df = filtered_df[filtered_df['name'].str.contains(search_name, case=False)]

# --- Build NetworkX Graph ---
G = nx.DiGraph()
for _, row in filtered_df.iterrows():
    G.add_node(row['partner_id'], label=row['name'], level=row['level'], join_date=row['join_date'])
for _, row in filtered_df.iterrows():
    if row['parent_id'] and row['parent_id'] in filtered_df['partner_id'].values:
        G.add_edge(row['parent_id'], row['partner_id'])

# --- Pyvis Network Visualization ---
net = Network(height="600px", width="100%", directed=True, notebook=False)
color_map = {'Distributor': '#1f77b4', 'Agent': '#2ca02c', 'Ambassador': '#d62728'}
for node, data in G.nodes(data=True):
    net.add_node(
        node,
        label=data['label'],
        title=f"Name: {data['label']}<br>Level: {data['level']}<br>Join Date: {data['join_date']}",
        color=color_map.get(data['level'], '#cccccc')
    )
for source, target in G.edges():
    net.add_edge(source, target)

# Enable node selection
net.toggle_physics(True)
net.show_buttons(filter_=['physics'])

# Save and display the network
with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as tmp_file:
    net.save_graph(tmp_file.name)
    tmp_path = tmp_file.name

st.components.v1.html(open(tmp_path, 'r', encoding='utf-8').read(), height=650)
os.remove(tmp_path)

# --- Node Detail Panel ---
st.subheader("Partner Details")
selected_id = st.number_input("Enter Partner ID to view details", min_value=int(df['partner_id'].min()), max_value=int(df['partner_id'].max()), step=1)
partner = df[df['partner_id'] == selected_id]
if not partner.empty:
    partner = partner.iloc[0]
    parent = df[df['partner_id'] == partner['parent_id']]['name'].values[0] if partner['parent_id'] else "None"
    children = df[df['parent_id'] == partner['partner_id']]['name'].tolist()
    st.markdown(f"""
    **Name:** {partner['name']}  
    **Level:** {partner['level']}  
    **Join Date:** {partner['join_date']}  
    **Revenue:** $[placeholder]  
    **Parent:** {parent}  
    **Children:** {', '.join(children) if children else 'None'}
    """)
else:
    st.info("Enter a valid Partner ID to see details.")